In [1]:
import ROOT as rt
import uproot
import numpy as np
from collections import OrderedDict
import csv
import os
import math
import sys
sys.path.append("/uscms/home/christiw/nobackup/CMSSW_11_3_4/src/MDS_CMSDAS2023/lib")

import importlib
import helper
importlib.reload(sys.modules['helper'])
from helper import  weight_calc, make_datacard

import tdrstyle
a = tdrstyle.setTDRStyle()

print(sys.version)

Welcome to JupyROOT 6.28/00
3.9.12 (main, Jun  7 2022, 16:09:12) 
[GCC 11.2.0]


# Load ntuples

In [2]:
fpath =OrderedDict()
tree = OrderedDict()

prod =['ggH']
decay = 'bbbb'



mass = [15, 40, 55]
if not decay == 'bbbb': mass = [7, 15, 40, 55]

OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm



path = '/uscms/home/christiw/nobackup/CMSSW_11_3_4/src/MDS_CMSDAS2023/ntuples/'

fpath['data'] = path + 'Run2_displacedJetMuonNtupler_V1p17_Data2018_17Sept2018_Run2018B-HighMET-17Sep2018_Job9_of_20.root'



for m in mass:
    for ct in OLD_CTAU:
        key = 'MC_'+str(m)+'_'+str(ct)                       
        fpath[key] = path +  'ggH_HToSSTobbbb_MH-125_MS-40_ctau-'+str(ct)+'_137000pb_weighted.root'

# fpath['hnl']= '/storage/af/user/christiw/HNL_electronType_ms5p0_plVe2_1e-5/HeavyNeutralLepton_Tree.root'                        
NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']


In [3]:
tree.keys()

odict_keys(['data', 'MC_15_100', 'MC_15_1000', 'MC_15_10000', 'MC_15_100000', 'MC_40_100', 'MC_40_1000', 'MC_40_10000', 'MC_40_100000', 'MC_55_100', 'MC_55_1000', 'MC_55_10000', 'MC_55_100000'])

# Selections

In [4]:
deltaPhi = {}
Nhits = {}
weight = {}
gLLP_ctau = {}


for k, T in tree.items():
    
    ########### SELECTION: CLUSTERS ############

    sel_rechitcluster = np.abs(T['cscRechitClusterEta'].array()) < 2.0

    me1112_veto = 10
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberPlus11'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberPlus12'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberMinus11'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterNRechitChamberMinus12'].array() <= me1112_veto)

    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitCluster_match_MB1Seg_0p4'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitCluster_match_RE12_0p4'].array() <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitCluster_match_RB1_0p4'].array() <= me1112_veto)

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T['cscRechitClusterJetVetoPt'].array() < 10)
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T['cscRechitClusterMuonVetoPt'].array() < 20)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T['cscRechitClusterTimeSpreadWeightedAll'].array() <= 20)

    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T['cscRechitClusterTimeWeighted'].array() < 12.5, \
                                                                         T['cscRechitClusterTimeWeighted'].array() > -5.0))

    ########### SELECTION: JETS ############

    sel_jet = np.logical_and(T['jetPt'].array() > 30, np.abs(T['jetEta'].array()) < 2.4 )
    sel_jet = np.logical_and(sel_jet, T['jetTightPassId'].array())

    ########### SELECTION: EVENTS ############
    # select only triggered events
    sel_ev = T['METNoMuTrigger'].array()

    sel_ev  = np.logical_and(sel_ev,np.sum(sel_rechitcluster,axis=1) >= 1)
    # sel_ev[k] = np.logical_and(sel_ev[k], (T['nDtRings')+T['nCscRings'))<10) # apply this in the ntuple before hand
    sel_ev = np.logical_and(sel_ev ,T['metEENoise'].array() > 200)
    sel_ev = np.logical_and(sel_ev , np.sum(sel_jet,axis=1)>=1)
    sel_ev = np.logical_and(sel_ev,T['Flag2_all'].array())


    ##### cut based ID ####
    cscRechitClusterNStation = T['cscRechitClusterNStation10'].array()[sel_rechitcluster][sel_ev][:,0]
    cscRechitClusterEta = T['cscRechitClusterEta'].array()[sel_rechitcluster][sel_ev][:,0]
    cscRechitClusterPhi = T['cscRechitClusterPhi'].array()[sel_rechitcluster][sel_ev][:,0]
    cscRechitClusterAvgStation = T['cscRechitClusterAvgStation10'].array()[sel_rechitcluster][sel_ev][:,0]
    cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation)==2, np.abs(cscRechitClusterEta) < 1.6)
    cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation)==3, np.abs(cscRechitClusterEta) < 1.6)
    cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation)==4, np.abs(cscRechitClusterEta) < 1.8)
    cond1 = np.logical_and(cscRechitClusterNStation==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation)==1, cond2), np.logical_or(cond3, cond4)))
    cond2 = np.logical_and(cscRechitClusterNStation > 1, np.abs(cscRechitClusterEta) < 1.9)
    bdt_sel = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))

    #### Save variables needed for ABCD ####                                   

    deltaPhi[k] = T['cscRechitClusterMetEENoise_dPhi'].array()[sel_rechitcluster][sel_ev][:,0][bdt_sel]
    Nhits[k] = T['cscRechitClusterSize'].array()[sel_rechitcluster][sel_ev][:,0][bdt_sel]
    weight[k] = T['weight'].array()[sel_ev][bdt_sel]
#                  *T.array('pileupWeight')*T.array('metSF'))[sel_ev][bdt_sel]
    gLLP_ctau[k] = T['gLLP_ctau'].array()[sel_ev][bdt_sel]
    print(k, np.sum(weight[k]))

data 2.0
MC_15_100 28.192163
MC_15_1000 3014.729
MC_15_10000 2594.8542
MC_15_100000 328.3366
MC_40_100 28.192163
MC_40_1000 3014.729
MC_40_10000 2594.8542
MC_40_100000 328.3366
MC_55_100 28.192163
MC_55_1000 3014.729
MC_55_10000 2594.8542
MC_55_100000 328.3366


# signal uncertainties

In [11]:
muonVeto = {}
cut_based_eff_unc = {}
clusterEff_unc = {}
higgsPtWeight = {}
JES = {}
pileup = {}
# ctau_reweight = {}
mc_stats = {}
lumi = {}
time_spread = {}

ggH_higgsPt = {}
ggH_qcdScale = {}
ggH_pdf = {}

cut_based_unc = [0.0921, 0.0852] # muon Eta reweighted
clustering_unc = [0.1506, 0.1047]

for m in mass:
    for ct in OLD_CTAU:
            key = 'MC_'+str(m)+'_'+str(ct)         

            lumi[key] = [0.018]*4
            muonVeto[key] = [0.045]*4
            time_spread[key] = [0.028]*4
            clusterEff_unc[key]  = [0.035] * 4 #shifting Nrechit cut from 130 to 135
            cut_based_eff_unc[key]=[0.051]*4 #muon eta reweighted
            JES[key] = [0.084,0.083,0.042,0.041]
            pileup[key] = [0.008, 0.0110, 0.0110, 0.0110]
            
            ggH_higgsPt[key] = [0.208, 0.205, 0.205,0.207,0.134, 0.133,0.133,0.134] #down*4/up*4
            ggH_qcdScale[key] = [0.067]*4+[0.046]*4 #down/up
            ggH_pdf[key] = [0.032]*4



sig_unc_temp = [lumi, muonVeto, time_spread, cut_based_eff_unc, clusterEff_unc, JES,pileup, \
           ggH_higgsPt, ggH_qcdScale, ggH_pdf]
sig_unc_name = ['lumi','muonVeto','time_spread','cut_based_eff_unc', 'clusterEff_unc', 'JES','pileup', \
                'ggH_higgsPt', 'ggH_qcdScale', 'ggH_pdf']


assert(len(sig_unc_temp)==len(sig_unc_name))

# make datacard

In [12]:

import importlib
import helper
importlib.reload(sys.modules['helper'])
from helper import  weight_calc, make_datacard

N_RECHIT_CUT = 130
DPHI_CUT = 0.75

ctaus = ['5','10','15','20','30','40', '50','60', '100', '125','150','200','300','500','600','700','800','900','1000', '2000','3000','4000', '5000', '6000','7000','8000','10000', '20000','30000','50000',\
         '100000', '200000', '300000', '500000', '1000000', '2000000', '3000000', '5000000', '6000000', '10000000'] #mm

bkg_unc = [] #ABCD closure, sum of stats from two VR
bkg_unc_name = []


bkg = []
bkg_unc = []

k = 'data'
a = np.count_nonzero(np.logical_and(Nhits[k] >= N_RECHIT_CUT, deltaPhi[k] < DPHI_CUT)) 
b = np.count_nonzero(np.logical_and(Nhits[k] >= N_RECHIT_CUT, deltaPhi[k] >= DPHI_CUT))
c = np.count_nonzero(np.logical_and(Nhits[k] < N_RECHIT_CUT, deltaPhi[k] >= DPHI_CUT)) 
d = np.count_nonzero(np.logical_and(Nhits[k] < N_RECHIT_CUT, deltaPhi[k] < DPHI_CUT)) 
observation= np.array([a,b,c,d])
print(observation)

#####
# start adding signal
#####
outDataCardsDir = '/uscms_data/d3/christiw/CMSSW_11_3_4/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/'
if not os.path.isdir(outDataCardsDir):os.makedirs(outDataCardsDir)
print(outDataCardsDir)
sig_norm = []
for m in mass:
    for ct in ctaus:
        modelName = 'ggH_HToSSTo'+decay+ '_mh125_mx'+str(m)+'_ctau'+str(ct)+'mm'

        
        signal_rate = []
        mc_stat_unc = []
        gmn = []
        sig_unc = []

        ctf = int(ct)
        ct0 = 10**math.ceil(math.log10(ctf))
        ct0 = max(ct0, OLD_CTAU[0])
        ct0 = min(ct0, OLD_CTAU[-1])
        k = 'MC_'+str(m)+'_'+str(ct0)
        weight_ctau = weight_calc(gLLP_ctau[k], int(ct)/10, int(ct0)/10) # convert everything to cm

        w = weight[k]*weight_ctau
        cond = np.logical_and(Nhits[k]>=N_RECHIT_CUT, deltaPhi[k]<DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
        
        cond = np.logical_and(Nhits[k]>=N_RECHIT_CUT, deltaPhi[k]>=DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
        
        cond = np.logical_and(Nhits[k]<N_RECHIT_CUT, deltaPhi[k]>=DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
        
        cond = np.logical_and(Nhits[k]<N_RECHIT_CUT, deltaPhi[k]<DPHI_CUT)
        signal_rate.append(np.sum(w[cond]))
              
        for j, ele in enumerate(sig_unc_temp):sig_unc.append(ele[k])
            
        make_datacard(outDataCardsDir, modelName, signal_rate, observation, sig_unc, sig_unc_name)


[0 0 2 4]
/uscms_data/d3/christiw/CMSSW_11_3_4/src/HiggsAnalysis/MuonSystemLimit/combine/datacards/
